In [2]:
import numpy as np
import pandas as pd
import yaml
import time
import importlib
from tqdm import tqdm
from loguru import logger
from functools import partial, wraps
from itertools import product
import os
from BF_zscore import BF_zscore
from EN import RPDE

In [3]:
base_dir = "../"

In [4]:
ts1 = np.loadtxt(base_dir + "ts1.txt")
ts2 = np.loadtxt(base_dir + "ts2.txt")
ts3 = np.loadtxt(base_dir + "ts3.txt")
ts4 = np.loadtxt(base_dir + "ts4.txt")

In [24]:
fpath = "../Configurations/basic.yaml"

In [4]:
def zscore_decorator(func):
    @wraps(func)
    def wrapper(y, *args, **kwargs):
        y = BF_zscore(y)
        return func(y, *args, **kwargs)
    return wrapper

In [5]:
def range_constructor(loader, node):
    start, end = loader.construct_sequence(node)
    return list(range(start, end+1))
yaml.add_constructor("!range", range_constructor)

In [6]:
def load_yaml2(file):
    print(f"Loading configuration file: {file.split('/')[-1]}")
    funcs = {}
    with open(file) as f:
        yf = yaml.load(f, Loader=yaml.FullLoader)

    for module_name in yf:
        print(f"\n*** Importing module {module_name} *** \n")
        module = importlib.import_module(module_name)
        for function_name in yf[module_name]:
            # Get the function's configuration dictionary
            function_config = yf[module_name][function_name]
            # If no configs section exists or if it's empty, use a list with single empty dict
            if ('configs' not in function_config or function_config.get('configs') is None or 
                function_config.get('configs') == []):
                configs = [{}]
            else:
                configs = function_config.get('configs', [{}])

            for params in configs:
                # Handle the case where params is None
                if params is None:
                    params = {}
                    
                zscore_first = params.pop("zscore", False)
                param_keys, param_vals = zip(*params.items()) if params else ([], [])
                
                param_combinations = [dict(zip(param_keys, values)) 
                                   for values in product(*[v if isinstance(v, list) 
                                                        else [v] for v in param_vals])]
                
                # If no parameter combinations were generated, add empty dict
                if not param_combinations:
                    param_combinations = [{}]
                
                # create a function for each parameter combination
                for param_set in param_combinations:
                    feature_name = (f"{module_name}_{function_name}_" + 
                                  "_".join(f"{v}" for k, v in param_set.items())
                                  if param_set else f"{module_name}_{function_name}")
                    if not zscore_first:
                        feature_name += "_raw"
                    
                    print(f"Adding operation {feature_name} with params {param_set} "
                          f"(Z-score={zscore_first})")
                    
                    base_func = partial(getattr(module, function_name), **param_set)
                    if zscore_first:
                        base_func = zscore_decorator(base_func)
                        
                    funcs[feature_name] = base_func
                    
    return funcs
            

In [7]:
# def load_yaml(file):
#     print(f"Loading configuration file: {file.split('/')[-1]}")
#     # print stats for configuration file
#     funcs = {}
#     with open(file) as f:
#         yf = yaml.load(f, Loader=yaml.FullLoader)
#         # instantiate the featuresß
#         for module_name in yf:
#             print(f"*** Importing module {module_name}")
#             module = importlib.import_module(module_name, __package__)
#             for function_name in yf[module_name]:
#                 configs = yf[module_name][function_name].get('configs', [{}])
#                 for params in configs:
#                     zscore_first = params.pop("zscore", False)

#                     param_keys, param_vals = zip(*params.items() if params else ([], []))
#                     param_combinations = [dict(zip(param_keys, values)) for values in product(*[
#                         v if isinstance(v, list) else [v] for v in param_vals])]
                    
#                     # create a function for each parameter combination
#                     for param_set in param_combinations:
#                         feature_name = f"{module_name}_{function_name}_" + "_".join(f"{v}" for k, v in param_set.items())
#                         print(f"Adding operation {feature_name} with params {param_set} (Z-score={zscore_first})")

#                         base_func = partial(getattr(module, function_name), **param_set)
#                         if zscore_first:
#                             base_func = zscore_decorator(base_func)
#                             #feature_func = partial(base_func, zscore=zscore_first, **param_set)
#                         funcs[feature_name] = base_func
#                         #funcs.append(feature_func)
#     return funcs

In [25]:
feats = load_yaml2(fpath)

Loading configuration file: basic.yaml

*** Importing module CO *** 

Adding operation CO_AutoCorr_1_Fourier with params {'tau': 1, 'method': 'Fourier'} (Z-score=True)
Adding operation CO_AutoCorr_2_Fourier with params {'tau': 2, 'method': 'Fourier'} (Z-score=True)
Adding operation CO_AutoCorr_3_Fourier with params {'tau': 3, 'method': 'Fourier'} (Z-score=True)
Adding operation CO_AutoCorr_4_Fourier with params {'tau': 4, 'method': 'Fourier'} (Z-score=True)
Adding operation CO_AutoCorr_5_Fourier with params {'tau': 5, 'method': 'Fourier'} (Z-score=True)
Adding operation CO_AutoCorr_6_Fourier with params {'tau': 6, 'method': 'Fourier'} (Z-score=True)
Adding operation CO_AutoCorr_7_Fourier with params {'tau': 7, 'method': 'Fourier'} (Z-score=True)
Adding operation CO_AutoCorr_8_Fourier with params {'tau': 8, 'method': 'Fourier'} (Z-score=True)
Adding operation CO_AutoCorr_9_Fourier with params {'tau': 9, 'method': 'Fourier'} (Z-score=True)
Adding operation CO_AutoCorr_10_Fourier with par

In [35]:
%%time
computed = {}
feature_count = 0
tstart = time.perf_counter()
for (feature, func) in zip(feats.keys(), feats.values()):
    computed[feature] = func(ts1)
    if (isinstance(computed[feature], float) or isinstance(computed[feature], int)):
        feature_count += 1
    else:
        feature_count += len(computed[feature]) 
telapsed = time.perf_counter() - tstart
print(f"Computed {len(computed)} operations ({feature_count} features)")
print(f"Time taken: {telapsed} seconds")

2025-02-19 09:36:26.944 | INFO     | DN:Quantile:219 - Using quantile p = 0.5 (median) by default
2025-02-19 09:36:27.633 | WARNING  | IN:_initialize_MI:316 - Number of nearest neighbors needs to be a string. Setting this for you...
2025-02-19 09:36:27.635 | WARNING  | IN:_initialize_MI:316 - Number of nearest neighbors needs to be a string. Setting this for you...
2025-02-19 09:36:27.636 | WARNING  | IN:_initialize_MI:316 - Number of nearest neighbors needs to be a string. Setting this for you...
2025-02-19 09:36:27.637 | WARNING  | IN:_initialize_MI:316 - Number of nearest neighbors needs to be a string. Setting this for you...
2025-02-19 09:36:27.638 | WARNING  | IN:_initialize_MI:316 - Number of nearest neighbors needs to be a string. Setting this for you...
2025-02-19 09:36:27.639 | WARNING  | IN:_initialize_MI:316 - Number of nearest neighbors needs to be a string. Setting this for you...
2025-02-19 09:36:27.682 | WARNING  | IN:_initialize_MI:316 - Number of nearest neighbors nee

This time series (N = 1000) is too short for StatAv(len,'500')
This time series (N = 1000) is too short for StatAv(len,'1000')
Computed 597 operations (2680 features)
Time taken: 2.0648555840016343 seconds
CPU times: user 2.2 s, sys: 192 ms, total: 2.39 s
Wall time: 2.06 s


unpack all of the individual features

In [105]:
individual_features = {}
for base_op in computed.keys():
    for (k, v) in zip(op.keys(), op.values()):
        feature_name = base_op + "." + k
        individual_features[feature_name] = float(v)

In [106]:
individual_features

{'CO_AutoCorr_1_Fourier.mean': 1361.7590925044824,
 'CO_AutoCorr_1_Fourier.median': 1354.4276774214243,
 'CO_AutoCorr_1_Fourier.std': 1167.8600687142855,
 'CO_AutoCorr_1_Fourier.range': 3907.5509097438953,
 'CO_AutoCorr_1_Fourier.proppos': 0.83,
 'CO_AutoCorr_1_Fourier.pcross': 0.05405405405405406,
 'CO_AutoCorr_1_Fourier.ac1': 0.979760851419583,
 'CO_AutoCorr_1_Fourier.ac10': 0.37712532445797964,
 'CO_AutoCorr_1_Fourier.ac50': 0.7619767031274244,
 'CO_AutoCorr_1_Fourier.tau': 7.983917503584886,
 'CO_AutoCorr_1_Fourier.finaldev': 898.492221526819,
 'CO_AutoCorr_2_Fourier.mean': 1361.7590925044824,
 'CO_AutoCorr_2_Fourier.median': 1354.4276774214243,
 'CO_AutoCorr_2_Fourier.std': 1167.8600687142855,
 'CO_AutoCorr_2_Fourier.range': 3907.5509097438953,
 'CO_AutoCorr_2_Fourier.proppos': 0.83,
 'CO_AutoCorr_2_Fourier.pcross': 0.05405405405405406,
 'CO_AutoCorr_2_Fourier.ac1': 0.979760851419583,
 'CO_AutoCorr_2_Fourier.ac10': 0.37712532445797964,
 'CO_AutoCorr_2_Fourier.ac50': 0.761976703127

In [103]:
len(individual_features)

6567

array, float, dict

In [81]:
tsdf = pd.DataFrame([individual_features])
tsdf.head(1)

,CO_AutoCorr_1_Fourier.mean,CO_AutoCorr_1_Fourier.median,CO_AutoCorr_1_Fourier.std,CO_AutoCorr_1_Fourier.range,CO_AutoCorr_1_Fourier.proppos,CO_AutoCorr_1_Fourier.pcross,CO_AutoCorr_1_Fourier.ac1,CO_AutoCorr_1_Fourier.ac10,CO_AutoCorr_1_Fourier.ac50,CO_AutoCorr_1_Fourier.tau,...,"MISC_ForcePotential_sine_[10, 0.04, 10].median","MISC_ForcePotential_sine_[10, 0.04, 10].std","MISC_ForcePotential_sine_[10, 0.04, 10].range","MISC_ForcePotential_sine_[10, 0.04, 10].proppos","MISC_ForcePotential_sine_[10, 0.04, 10].pcross","MISC_ForcePotential_sine_[10, 0.04, 10].ac1","MISC_ForcePotential_sine_[10, 0.04, 10].ac10","MISC_ForcePotential_sine_[10, 0.04, 10].ac50","MISC_ForcePotential_sine_[10, 0.04, 10].tau","MISC_ForcePotential_sine_[10, 0.04, 10].finaldev"
0,1361.759093,1354.427677,1167.860069,3907.55091,0.83,0.054054,0.979761,0.377125,0.761977,7.983918,...,1354.427677,1167.860069,3907.55091,0.83,0.054054,0.979761,0.377125,0.761977,7.983918,898.492222


In [102]:
np.count_nonzero(np.isnan(tsdf.iloc[0].values)) # number of NaN values 

0

In [11]:
class Calculator:
    """Compute all univariate time series features.
    
    The calculator takes in a univariate time-series dataset of N instances and returns a 
    feature matrix of size N x F where F is the number of features.

    """

    def __init__(self, dataset=None, name=None, configfile=None):
        
        # define a configfile by sb
        self._features = {} 

        def compute(self):
            pass 

        def load_yaml(file : str) -> dict:
            # function to construct the partials from the YAML file. 

            print(f"Loading configuration file: {file.split('/')[-1]}")
            funcs = {} # dictionary of partial functions to be re-used. 
            with open(file) as f:
                yf = yaml.load(f, Loader=yaml.FullLoader)

            for module_name in yf:
                print(f"\n*** Importing module {module_name} *** \n")
                module = importlib.import_module(module_name)
                for function_name in yf[module_name]:
                    # Get the function's configuration dictionary
                    function_config = yf[module_name][function_name]
                    # If no configs section exists or if it's empty, use a list with single empty dict
                    if ('configs' not in function_config or function_config.get('configs') is None or 
                        function_config.get('configs') == []):
                        configs = [{}]
                    else:
                        configs = function_config.get('configs', [{}])

                    for params in configs:
                        # Handle the case where params is None
                        if params is None:
                            params = {}
                            
                        zscore_first = params.pop("zscore", False)
                        param_keys, param_vals = zip(*params.items()) if params else ([], [])
                        
                        param_combinations = [dict(zip(param_keys, values)) 
                                        for values in product(*[v if isinstance(v, list) 
                                                                else [v] for v in param_vals])]
                        # If no parameter combinations were generated, add empty dict
                        if not param_combinations:
                            param_combinations = [{}]
                        
                        # create a function for each parameter combination
                        for param_set in param_combinations:
                            feature_name = (f"{module_name}_{function_name}_" + 
                                        "_".join(f"{v}" for k, v in param_set.items())
                                        if param_set else f"{module_name}_{function_name}")
                            if not zscore_first:
                                feature_name += "_raw"
                            
                            print(f"Adding operation {feature_name} with params {param_set} "
                                f"(Z-score={zscore_first})")
                            
                            base_func = partial(getattr(module, function_name), **param_set)
                            if zscore_first:
                                base_func = zscore_decorator(base_func)
                                
                            funcs[feature_name] = base_func
                            
            return funcs
        
